In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datasets

In [3]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
Tesla T4


In [5]:
df1 = pd.read_pickle('/content/drive/MyDrive/data/trimmed_processed/WELFake.pkl')
df2 = pd.read_pickle('/content/drive/MyDrive/data/trimmed_processed/FAKE-REAL.pkl')
df = pd.concat([df1,df2])

df['label'] = df['label'].map({'FAKE':0, 'REAL':1})

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'],df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 256

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length = max_length)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length = max_length)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {
            "labels": torch.tensor(self.labels[idx])
        }

train_dataset = NewsDataset(train_encodings, list(train_labels))
val_dataset = NewsDataset(val_encodings, list(val_labels))


In [8]:
model3 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/data/results/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # keep small
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,  # mixed precision for faster training
)

print(training_args.device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda:0


In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [10]:
trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kknaheel16 (kknaheel16-mes-college-of-engineering-kuttipuram) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.322700,0.300673,0.804076,0.797476,0.849612,0.804076
2,0.268900,0.300568,0.812278,0.806466,0.854957,0.812278
3,0.258800,0.302799,0.807536,0.800928,0.854576,0.807536


TrainOutput(global_step=35112, training_loss=0.300822224098774, metrics={'train_runtime': 4700.2002, 'train_samples_per_second': 59.758, 'train_steps_per_second': 7.47, 'total_flos': 3.695065883712e+16, 'train_loss': 0.300822224098774, 'epoch': 3.0})

In [11]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.3005683720111847, 'eval_accuracy': 0.8122783782629128, 'eval_f1': 0.8064659289252148, 'eval_precision': 0.8549574719662312, 'eval_recall': 0.8122783782629128, 'eval_runtime': 90.7906, 'eval_samples_per_second': 257.813, 'eval_steps_per_second': 32.228, 'epoch': 3.0}


In [12]:
trainer.save_model("../../model/2dataset-bert/")
tokenizer.save_pretrained("../../model/2dataset-bert/")


('../../model/3dataset-bert/tokenizer_config.json',
 '../../model/3dataset-bert/special_tokens_map.json',
 '../../model/3dataset-bert/vocab.txt',
 '../../model/3dataset-bert/added_tokens.json')